In [ ]:

inputFile = "data/subdivision1.txt"
file = open(inputFile, "r")
input = file.readlines()
file.close()
# print(input)


In [ ]:
import re
from DoublyConnectedEdgeList import Vertex, Face, HalfEdge

vertex = []
face = []
halfedge = []

def readVertex(line):
	idx = re.search('v(.+?) ', line).group(1)
	x = re.search(' \((.+?),', line).group(1)
	y = re.search(', (.+?)\)', line).group(1)
	es = re.search(' e(.+?),', line).group(1)
	ee = re.search('e\d,(.+?)\n', line).group(1)
	v = Vertex(idx, x, y)
	vertex.append(v)
	return v

def readFace(line):
	idx = re.search('f(\d) ', line).group(1)
	f = Face(idx)
	face.append(f)
	return f

def readHalfEdge(line):
	vsi = re.search('e(\d),\d v', line).group(1)
	vei = re.search('e\d,(\d) v', line).group(1)
	fi = re.search('f(\d) ', line).group(1)

	vs = vertex[int(vsi)-1]
	ve = vertex[int(vei)-1]
	f = face[int(fi)-1]
	e = HalfEdge(vs, ve, f)
	halfedge.append(e)
	return e

# find x,y cordinates of vertex for ploting
x_values = []
y_values = []

for line in input:
	c = line[0]
	if c == 'v':
		v = readVertex(line)
		x_values.append(float(v.x))
		y_values.append(float(v.y))
	elif c == 'f':
		f = readFace(line)
	elif c == 'e':
		e = readHalfEdge(line)
	pass

# the face outside bounding box
face_out = Face(str(len(face)))

# bounding vertice
offset = 5
topl = Vertex(len(vertex)+1, min(x_values)-offset, max(y_values)+offset)
topr = Vertex(len(vertex)+2, max(x_values)+offset, max(y_values)+offset)
botr = Vertex(len(vertex)+3, max(x_values)+offset, min(y_values)-offset)
botl = Vertex(len(vertex)+4, min(x_values)-offset, min(y_values)-offset)

# bounding edges
halfedge[len(halfedge):] = [
	HalfEdge(topl, topr, face_out), 
	HalfEdge(topr, botr, face_out), 
	HalfEdge(botr, botl, face_out), 
	HalfEdge(botl, topl, face_out)
	]

# vertex[len(vertex):] = [topl, topr, botr, botl]

In [ ]:
# for v in vertex:
# 	print(v)

# for f in face:
# 	print(f)
	
# for e in halfedge:
# 	print(e)

# print(x_values)
# print(y_values)

from DirectedAcyclicGraph import Point, LineSegment
seg_edg = dict()

'''Append multiple values to a key in the given dictionary'''
def add_values_to_dict(target_dict, key, list_of_values):
	if key not in target_dict:
		target_dict[key] = list()
	target_dict[key].extend(list_of_values)
	return target_dict


ls = []
s_e_dict = dict()
for e in halfedge:
	seg = e.toLineSegment()	#TODO new seg every iter
	ls.append(seg)
	s_e_dict = add_values_to_dict(s_e_dict, seg, [e])

print(s_e_dict)

edg1 = halfedge[0]
edg2 = halfedge[1]
seg1 = edg1.toLineSegment()
seg2 = edg2.toLineSegment()

print('edge1', edg1, 'edge2', edg2)
print(s_e_dict[ls[0]])
print(edg1.__ne__, edg2.__ne__)

for s in ls:
	print(s)

In [ ]:
# Create a dictionary where multiple
# values are associated with a key
word_freq = {'is'  : [1, 3, 4, 8, 10],
			'at'   : [3, 10, 15, 7, 9],
			'test' : [5, 3, 7, 8, 1],
			'this' : [2, 3, 5, 6, 11],
			'why'  : [10, 3, 9, 8, 12] }
# Get multiple values of a key as list
value_list = word_freq['at']
print('Values of key "at" are:')
print(value_list)

def add_values_in_dict(sample_dict, key, list_of_values):
	''' Append multiple values to a key in the given dictionary '''
	if key not in sample_dict:
		sample_dict[key] = list()
	sample_dict[key].extend(list_of_values)
	return sample_dict

# Append multiple values for existing key 'at'
word_freq = add_values_in_dict(word_freq, 'at', [20, 21, 22])
print('Contents of the dictionary: ')
print(word_freq['at'])

In [ ]:
import pylab as pl
import matplotlib.pyplot as plt
import matplotlib.collections as mc

segment = []
for i in range(1, len(halfedge)):
	e = halfedge[i]
	stt = (e.vs.x, e.vs.y)
	end = (e.ve.x, e.ve.y)
	seg = [stt, end]
	segment.append(seg)

fig, ax = pl.subplots()

ax.add_collection(mc.LineCollection(segment[:-4], colors=('blue')))
ax.add_collection(mc.LineCollection(segment[-4:], colors=('black')))
ax.set_xlim(0, 10)

ax.autoscale()
ax.margins(0.1)

for v in vertex:
	plt.text(v.x-0.015, v.y+0.25, "v"+str(v.idx))

plt.scatter(x_values, y_values, color="red", s=50)
plt.show()

In [ ]:
import pylab as pl
import matplotlib.pyplot as plt
import matplotlib.collections as mc

segment = []
for i in range(1, len(halfedge)):
	e = halfedge[i]
	stt = (e.vs.x, e.vs.y)
	end = (e.ve.x, e.ve.y)
	seg = [stt, end]
	segment.append(seg)

fig, ax = pl.subplots()

ax.add_collection(mc.LineCollection(segment[:-4], colors=('blue')))
ax.add_collection(mc.LineCollection(segment[-4:], colors=('black')))
ax.set_xlim(0, 10)

ax.autoscale()
ax.margins(0.1)

for v in vertex:
	plt.text(v.x-0.015, v.y+0.25, "v"+str(v.idx))

plt.scatter(x_values, y_values, color="red", s=50)
plt.show()

In [ ]:
# Algorithm TRAPEZOIDALMAP(S)
# Input. A set S of n non-crossing line segments.
# Output. The trapezoidal map T(S) and a search structure D for T(S) in a bounding box.
# 1. Determine a bounding box R that contains all segments of S, and initialize the trapezoidal map structure T and search structure D for it.
# 2. Compute a random permutation s1,s2,...,sn of the elements of S.
# 3. for i ← 1 to n
# 4. do Find the set ∆0,∆1,...,∆k of trapezoids in T properly intersected by si.
# 5. Remove ∆0,∆1,...,∆k from T and replace them by the new trapezoids that appear because of the insertion of si.
# 6. Remove the leaves for ∆0,∆1,...,∆k from D, and create leaves for the new trapezoids. Link the new leaves to the existing inner nodes by adding some new inner nodes, as explained below.


In [ ]:
# Algorithm FOLLOWSEGMENT(T,D,si)
# Input. A trapezoidal map T, a search structure D for T, and a new segment si.
# Output. The sequence ∆0,...,∆k of trapezoids intersected by si.
# 1. Let p and q be the left and right endpoint of si.
# 2. Search with p in the search structure D to find ∆0.
# 3. j ← 0;
# 4. while q lies to the right of rightp(∆j)
# 5. do if rightp(∆j) lies above si
# 6. then Let ∆j+1 be the lower right neighbor of ∆j.
# 7. else Let ∆j+1 be the upper right neighbor of ∆j.
# 8. j ← j +1
# 9. return ∆0,∆1,...,∆j


In [ ]:
# Algorithm RANDOMPERMUTATION(A)
# Input. An array A[1···n].
# Output. The array A[1···n] with the same elements, but rearranged into a random permutation.
# 1. for k ← n downto 2
# 2. do rndindex ←RANDOM(k)
# 3. Exchange A[k] and A[rndindex].

def randomSegment():
	halfedge
	pass


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

g = nx.Graph()

g.add_edge(1, 2)
g.add_edge(2, 3)
g.add_edge(3, 4)
g.add_edge(1, 4)
g.add_edge(1, 5)
g.add_edge(5, 6)
g.add_edge(5, 7)
g.add_edge(4, 8)
g.add_edge(3, 8)

# drawing in circular layout
nx.draw_circular(g, with_labels = True)
plt.savefig("filename1.png")
# clearing the current plot
plt.clf()

# drawing in planar layout
nx.draw_planar(g, with_labels = True)
plt.savefig("filename2.png")
# clearing the current plot
plt.clf()

# drawing in random layout
nx.draw_random(g, with_labels = True)
plt.savefig("filename3.png")
# clearing the current plot
plt.clf()

# drawing in spectral layout
nx.draw_spectral(g, with_labels = True)
plt.savefig("filename4.png")
# clearing the current plot
plt.clf()

# drawing in spring layout
nx.draw_spring(g, with_labels = True)
plt.savefig("filename5.png")
# clearing the current plot
plt.clf()

# drawing in shell layout
nx.draw_shell(g, with_labels = True)
plt.savefig("filename6.png")
# clearing the current plot
plt.clf()

In [ ]:
import numpy as np
import pylab as pl
from matplotlib import collections as mc

lines = [[(0, 1), (1, 1)], [(2, 3), (3, 3)], [(1, 2), (1, 3)]]
c = np.array([(1, 0, 0, 1), (0, 1, 0, 1), (0, 0, 1, 1)])
lc = mc.LineCollection(lines, colors=c, linewidths=2)

fig, ax = pl.subplots()
ax.add_collection(lc)
ax.autoscale()
ax.margins(0.1)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

graph = {
	's1': ['v', 't1','w'],
	's2': ['t1','s1'],
	's3': ['v','w'],
	's4': ['x','y'],
	'x': ['v','w'],
	'v': ['t1', 'w'],
	'w': ['y','t1','t2'],
	'y': ['v','t1','t2'],
	't1': [],
	't2': []
	}

def main(G):
	fig = plt.figure()
	fig.show()

	graph = nx.DiGraph()

	for v in G.keys():
		graph.add_node(v)

	for delta in G.items():
		for w in delta[1]:
			graph.add_edge(delta[0],w)

	#posit = nx.shell_layout(G) #ISN'T NEEDED ANYMORE

	nx.draw_planar(graph,with_labels = True, alpha=0.8) #NEW FUNCTION
	fig.canvas.draw()

main(graph)